<a href="https://colab.research.google.com/github/arturomf94/pysyft-crash-course/blob/master/notebooks/comprehensive_pysyft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Intro**

![alt text](https://github.com/arturomf94/pysyft-crash-course/raw/master/docs/openmined.png)

In [0]:
!pip install 'syft[udacity]'

## PySyft

**PySyft** is a tool for privacy-preserving, decentralized deep learning. 

* This is a comprehensive, step-by-step guide to the basic concepts in PySyft. Most of the code here is originally from [the tutorials](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials). A detailed explanation of the concepts treated here can be found there, along with extra resources.

* This tutorial covers the basics of remote execution, federated learning, and SMPC.

## Motivation

* New regulations, such as the [GDPR](https://eugdpr.org/), imply that working with (personal) data will be increasingly hard for anyone. 

* **Privacy matters** - it is a fundamental value to have in a society, but lot of modern technologies disregard this. PySyft, along with many other projects, are tools that will facilitate privacy-preserving technologies.

# Basics

## Setup

In [0]:
import sys
import torch
from torch.nn import Parameter
import torch.nn as nn
import torch.nn.functional as F
import syft as sy
hook = sy.TorchHook(torch)

In [4]:
torch.tensor([1,2,3,4,5]) # A normal PyTorch tensor

tensor([1, 2, 3, 4, 5])

## Remote Tensors

*How could we train a model with data we don't have access to?*

We would need some notion of **remote execution**.

In [5]:
# Normal operation with tensors
x = torch.tensor([1,2,3,4,5])
y = x + x
print(y)

tensor([ 2,  4,  6,  8, 10])


In [0]:
# We use pointers, instead of local tensors!
bob = sy.VirtualWorker(hook, id="bob")

In [0]:
# Create some data
x = torch.tensor([1,2,3,4,5])
y = torch.tensor([1,1,1,1,1])

In [0]:
# Send this to Bob...
x_ptr = x.send(bob)
y_ptr = y.send(bob)

In [9]:
x_ptr

(Wrapper)>[PointerTensor | me:16775580167 -> bob:34070221148]

In [10]:
bob._objects

{2432983751: tensor([1, 1, 1, 1, 1]), 34070221148: tensor([1, 2, 3, 4, 5])}

In [0]:
# We can also perform operations with these pointers
z = x_ptr + x_ptr

In [12]:
z

(Wrapper)>[PointerTensor | me:23622416525 -> bob:74017461552]

In [13]:
bob._objects # Bob has z now!

{2432983751: tensor([1, 1, 1, 1, 1]),
 34070221148: tensor([1, 2, 3, 4, 5]),
 74017461552: tensor([ 2,  4,  6,  8, 10])}

In [14]:
x_ptr

(Wrapper)>[PointerTensor | me:16775580167 -> bob:34070221148]

A pointer has the following metadata: 

* `x_ptr.location : bob` the location of the data the pointer is pointing to.

* `x_ptr.id_at_location: <random integer>` the id of the tensor.

  Format: `<id_at_location>@<location>`

* `x_ptr.id : <random integer>` the id of our pointer.
* `x_ptr.owner : "me"` the owner of that pointer.

In [15]:
x_ptr.location

<VirtualWorker id:bob #objects:3>

In [16]:
bob

<VirtualWorker id:bob #objects:3>

In [17]:
bob == x_ptr.location

True

In [18]:
x_ptr.id_at_location

34070221148

In [19]:
x_ptr.owner

<VirtualWorker id:me #objects:0>

In [20]:
me = sy.local_worker
me # local worker created with sy.TorchHook()

<VirtualWorker id:me #objects:0>

In [21]:
me == x_ptr.owner

True

In [22]:
x_ptr

(Wrapper)>[PointerTensor | me:16775580167 -> bob:34070221148]

In [23]:
x_ptr.get()

tensor([1, 2, 3, 4, 5])

In [24]:
y_ptr

(Wrapper)>[PointerTensor | me:28142303267 -> bob:2432983751]

In [25]:
y_ptr.get()

tensor([1, 1, 1, 1, 1])

In [26]:
z.get()

tensor([ 2,  4,  6,  8, 10])

In [27]:
# The .get() operator allows us to get back our data
bob._objects

{}

## Pointer Operations

In [0]:
x = torch.tensor([1,2,3,4,5]).send(bob)
y = torch.tensor([1,1,1,1,1]).send(bob)

In [0]:
z = x + y

In [30]:
z

(Wrapper)>[PointerTensor | me:64969019240 -> bob:97499167153]

In [31]:
z.get()

tensor([2, 3, 4, 5, 6])

We can use all of Torch's operations:

In [32]:
x

(Wrapper)>[PointerTensor | me:65779606657 -> bob:83291032525]

In [33]:
y

(Wrapper)>[PointerTensor | me:20152843349 -> bob:94670693689]

In [34]:
z = torch.add(x,y); z

(Wrapper)>[PointerTensor | me:24483564965 -> bob:97289510165]

In [35]:
z.get()

tensor([2, 3, 4, 5, 6])

In [0]:
x = torch.tensor([1,2,3,4,5.], requires_grad=True).send(bob)
y = torch.tensor([1,1,1,1,1.], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()

In [38]:
z.backward()

(Wrapper)>[PointerTensor | me:93529277569 -> bob:84526282541]

In [0]:
x = x.get()

In [40]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [41]:
x.grad

tensor([1., 1., 1., 1., 1.])

In [0]:
%reset -f

# Federated Learning

Remote execution is very cool and it lays the foundation for one of the most important tools for private ML: federated learning. 

Essentially, federated learning allows us to take the models where the data is generated instead of taking the data to a central server. This is at the heart of **decentralized AI**.

![alt text](https://raw.githubusercontent.com/arturomf94/pysyft-crash-course/master/docs/centralized_learning.png)

![alt text](https://raw.githubusercontent.com/arturomf94/pysyft-crash-course/master/docs/federated_learning.png)

## Intro

First, let's outline a traditional training procedure...

In [0]:
import torch
from torch import nn
from torch import optim

In [0]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]])
target = torch.tensor([[0],[0],[1],[1.]])

# A Toy Model
model = nn.Linear(2,1)

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)

In [45]:
train()

tensor(4.1121)
tensor(1.7416)
tensor(1.1922)
tensor(0.8795)
tensor(0.6562)
tensor(0.4917)
tensor(0.3695)
tensor(0.2784)
tensor(0.2103)
tensor(0.1591)
tensor(0.1206)
tensor(0.0915)
tensor(0.0695)
tensor(0.0529)
tensor(0.0403)
tensor(0.0307)
tensor(0.0234)
tensor(0.0178)
tensor(0.0136)
tensor(0.0104)


Now let's do the same but in a federated way...

In [0]:
import syft as sy
hook = sy.TorchHook(torch)

In [0]:
# create a couple workers
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [0]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
data_bob = data[0:2]
target_bob = target[0:2]

data_alice = data[2:]
target_alice = target[2:]

# Iniitalize A Toy Model
model = nn.Linear(2,1)

data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)
target_bob = target_bob.send(bob)
target_alice = target_alice.send(alice)

# organize pointers into a list
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

In [0]:
from syft.federated.floptimizer import Optims
workers = ['bob', 'alice']
optims = Optims(workers, optim=optim.Adam(params=model.parameters(),lr=0.1))

In [0]:
def train():
    # Training Logic
    for iter in range(10):
        
        # NEW) iterate through each worker's dataset
        for data,target in datasets:
            # NEW) send model to correct worker
            model.send(data.location)   
            #Call the optimizer for the worker using get_optim
            opt = optims.get_optim(data.location.id)
            #print(data.location.id)
            # 1) erase previous gradients (if they exist)
            opt.zero_grad()
            # 2) make a prediction
            pred = model(data)
            # 3) calculate how much we missed
            loss = ((pred - target)**2).sum()
            # 4) figure out which weights caused us to miss
            loss.backward()
            # 5) change those weights
            opt.step()
            # NEW) get model (with gradients)
            model.get()
            # 6) print our progress
            print(loss.get()) # NEW) slight edit... need to call .get() on loss\
    
# federated averaging

In [51]:
train()

tensor(1.7038, requires_grad=True)
tensor(0.1269, requires_grad=True)
tensor(0.7363, requires_grad=True)
tensor(0.2842, requires_grad=True)
tensor(0.5118, requires_grad=True)
tensor(0.2872, requires_grad=True)
tensor(0.4097, requires_grad=True)
tensor(0.2263, requires_grad=True)
tensor(0.3554, requires_grad=True)
tensor(0.1477, requires_grad=True)
tensor(0.3224, requires_grad=True)
tensor(0.0776, requires_grad=True)
tensor(0.2966, requires_grad=True)
tensor(0.0290, requires_grad=True)
tensor(0.2688, requires_grad=True)
tensor(0.0050, requires_grad=True)
tensor(0.2323, requires_grad=True)
tensor(0.0001, requires_grad=True)
tensor(0.1854, requires_grad=True)
tensor(0.0044, requires_grad=True)


In [0]:
%reset -f

## Aggregation

In the previous example we directly got the model back from the workers at each iteration with the `get()` operator.

This poses a privacy problem, since, although we don't have direct access to the data, we can learn a lot with the gradients!

To solve this we could average the gradient across the workers **before** getting the model back.

### Advanced Remote Execution

In [0]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

In [0]:
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')

In [55]:
# this is a local tensor
x = torch.tensor([1,2,3,4]); x

tensor([1, 2, 3, 4])

In [56]:
# this sends the local tensor to Bob
x_ptr = x.send(bob)

# this is now a pointer
x_ptr

(Wrapper)>[PointerTensor | me:57286092926 -> bob:91021715272]

In [57]:
# now we can SEND THE POINTER to alice!!!
pointer_to_x_ptr = x_ptr.send(alice)

pointer_to_x_ptr

(Wrapper)>[PointerTensor | me:50696024554 -> alice:57286092926]

In [58]:
# As you can see above, Bob still has the actual data (data is always stored in a LocalTensor type). 
bob._objects

{91021715272: tensor([1, 2, 3, 4])}

In [59]:
# and we can use .get() to get x_ptr back from Alice
x_ptr = pointer_to_x_ptr.get()
x_ptr

(Wrapper)>[PointerTensor | me:57286092926 -> bob:91021715272]

In [60]:
# and then we can use x_ptr to get x back from Bob!
x = x_ptr.get()
x

tensor([1, 2, 3, 4])

In [61]:
bob._objects

{}

In [62]:
alice._objects

{}

In [0]:
p2p2x = torch.tensor([1,2,3,4,5]).send(bob).send(alice)

y = p2p2x + p2p2x

In [64]:
bob._objects

{41015732092: tensor([ 2,  4,  6,  8, 10]),
 89762003190: tensor([1, 2, 3, 4, 5])}

In [65]:
alice._objects

{79582926382: (Wrapper)>[PointerTensor | alice:79582926382 -> bob:41015732092],
 99812457942: (Wrapper)>[PointerTensor | alice:99812457942 -> bob:89762003190]}

In [66]:
y.get().get()

tensor([ 2,  4,  6,  8, 10])

In [67]:
bob._objects

{89762003190: tensor([1, 2, 3, 4, 5])}

In [68]:
alice._objects

{99812457942: (Wrapper)>[PointerTensor | alice:99812457942 -> bob:89762003190]}

In [69]:
p2p2x.get().get()

tensor([1, 2, 3, 4, 5])

In [70]:
bob._objects

{}

In [71]:
alice._objects

{}

In [0]:
# x is now a pointer to the data which lives on Bob's machine
x = torch.tensor([1,2,3,4,5]).send(bob)

In [73]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

  bob: {42893084212: tensor([1, 2, 3, 4, 5])}
alice: {}


In [0]:
x = x.move(alice)

In [75]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

  bob: {}
alice: {42893084212: tensor([1, 2, 3, 4, 5])}


In [76]:
x

(Wrapper)>[PointerTensor | me:29578883901 -> alice:42893084212]

In [0]:
%reset -f

## Trusted Aggregator

With advanced remote execution techniques we can now have a reliable / trusted third-party to aggregate the gradients before sending the model back to us.



In [0]:
import torch
import syft as sy
import copy
hook = sy.TorchHook(torch)
from torch import nn, optim

In [0]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")


# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [0]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)

In [0]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

In [82]:
for i in range(10):
    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()
    bobs_opt.step()
    bobs_loss = bobs_loss.get().data
    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()
    alices_opt.step()
    alices_loss = alices_loss.get().data  
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.8326) Alice:tensor(7.4444)
Bob:tensor(0.2031) Alice:tensor(0.0576)
Bob:tensor(0.0581) Alice:tensor(0.0007)
Bob:tensor(0.0235) Alice:tensor(0.0002)
Bob:tensor(0.0141) Alice:tensor(0.0002)
Bob:tensor(0.0107) Alice:tensor(0.0002)
Bob:tensor(0.0088) Alice:tensor(0.0001)
Bob:tensor(0.0074) Alice:tensor(0.0001)
Bob:tensor(0.0063) Alice:tensor(8.9752e-05)
Bob:tensor(0.0054) Alice:tensor(7.4701e-05)


In [0]:
alices_model.move(secure_worker)

In [0]:
bobs_model.move(secure_worker)

In [0]:
with torch.no_grad():
    model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

In [86]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)
    bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)
    for wi in range(worker_iters):
        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target)**2).sum()
        bobs_loss.backward()
        bobs_opt.step()
        bobs_loss = bobs_loss.get().data
        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target)**2).sum()
        alices_loss.backward()
        alices_opt.step()
        alices_loss = alices_loss.get().data 
    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)
    with torch.no_grad():
        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0075) Alice:tensor(0.0028)
Bob:tensor(0.0073) Alice:tensor(0.0019)
Bob:tensor(0.0066) Alice:tensor(0.0011)
Bob:tensor(0.0056) Alice:tensor(0.0006)
Bob:tensor(0.0047) Alice:tensor(0.0004)
Bob:tensor(0.0038) Alice:tensor(0.0002)
Bob:tensor(0.0031) Alice:tensor(0.0001)
Bob:tensor(0.0024) Alice:tensor(9.2327e-05)
Bob:tensor(0.0019) Alice:tensor(6.2528e-05)
Bob:tensor(0.0015) Alice:tensor(4.3476e-05)


In [0]:
preds = model(data)
loss = ((preds - target) ** 2).sum()

In [88]:
print(preds)
print(target)
print(loss.data)

tensor([[0.1101],
        [0.0909],
        [0.8853],
        [0.8661]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.0515)


In [0]:
%reset -f

# Encrypted Programs

A **trusted aggregator** is a very strong (and naive) assumption. Often times we cannot find trusted third-parties. One (almost magical) solution to this is to use Secure Multi-Party Computation (SMPC), an algorithm that let's us perform **encrypted computation**.

**The idea** is that, unlike in public-key cryptography, each agent will not have a public/private key. Instead, each will have a *share* of the value to be encrypted. In a sense, each agent has **a** private key, but they all need to be used to decrypt.

In [0]:
Q = 1234567891011

In [0]:
x = 25

In [0]:
import random

def encrypt(x):
    share_a = random.randint(-Q,Q)
    share_b = random.randint(-Q,Q)
    share_c = (x - share_a - share_b) % Q
    return (share_a, share_b,  share_c)

In [0]:
encrypt(x)

(-520169851765, -324163672501, 844333524291)

In [0]:
def decrypt(*shares):
    return sum(shares) % Q

In [0]:
a,b,c = encrypt(25)

In [95]:
decrypt(a, b, c)

25

In [97]:
decrypt(a, b)

295264354699

### Basic Arithmetic:

In [0]:
x = encrypt(25)
y = encrypt(5)

In [0]:
def add(x, y):
    z = list()
    # the first worker adds their shares together
    z.append((x[0] + y[0]) % Q)
    
    # the second worker adds their shares together
    z.append((x[1] + y[1]) % Q)
    
    # the third worker adds their shares together
    z.append((x[2] + y[2]) % Q)
    
    return z

In [100]:
decrypt(*add(x,y))

30

In [0]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
bill = sy.VirtualWorker(hook, id="bill")

In [0]:
x = torch.tensor([25])

In [103]:
x

tensor([25])

In [0]:
encrypted_x = x.share(bob, alice, bill)

In [105]:
encrypted_x.get()

tensor([25])

In [106]:
bob._objects

{}

In [0]:
x = torch.tensor([25]).share(bob, alice, bill)

In [108]:
# Bob's share
bobs_share = list(bob._objects.values())[0]
bobs_share

tensor([1138234187563384712])

In [109]:
# Alice's share
alices_share = list(alice._objects.values())[0]
alices_share

tensor([3250733381822143904])

In [110]:
# Bill's share
bills_share = list(bill._objects.values())[0]
bills_share

tensor([222718449041859313])

In [111]:
Q = x.child.field

(bobs_share + alices_share + bills_share) % Q

tensor([25])

In [0]:
x = torch.tensor([25]).share(bob,alice)
y = torch.tensor([5]).share(bob,alice)

In [113]:
z = x + y
z.get()

tensor([30])

In [114]:
z = x - y
z.get()

tensor([20])

### Multiplication

In [0]:
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")

In [0]:
x = torch.tensor([25]).share(bob, alice, crypto_provider=crypto_provider)
y = torch.tensor([5]).share(bob, alice, crypto_provider=crypto_provider)

In [117]:
# multiplication

z = x * y
z.get()

tensor([125])

In [0]:
x = torch.tensor([[1, 2],[3,4]]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([[2, 0],[0,2]]).share(bob,alice, crypto_provider=crypto_provider)

In [119]:
# matrix multiplication

z = x.mm(y)
z.get()

tensor([[2, 4],
        [6, 8]])

### Comparison

In [0]:
x = torch.tensor([25]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([5]).share(bob,alice, crypto_provider=crypto_provider)

In [121]:
z = x > y
z.get()

tensor([1])

In [122]:
z = x <= y
z.get()

tensor([0])

In [123]:
z = x == y
z.get()

tensor([0])

In [124]:
z = x == y + 20
z.get()

tensor([1])

In [125]:
x = torch.tensor([2, 3, 4, 1]).share(bob,alice, crypto_provider=crypto_provider)
x.max().get()

tensor([4])

In [126]:
x = torch.tensor([[2, 3], [4, 1]]).share(bob,alice, crypto_provider=crypto_provider)
max_values, max_ids = x.max(dim=0)
max_values.get()

tensor([4, 3])

In [0]:
%reset -f

# Example

We can now use Federated Learning + SMPC to have a privacy-preserving ML framework. The following is an example of what can be done!

## Encrypted NN on Encrypted Data

![alt text](https://raw.githubusercontent.com/arturomf94/pysyft-crash-course/master/docs/mnist.png)

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import syft as sy

In [0]:
# Set everything up
hook = sy.TorchHook(torch) 

alice = sy.VirtualWorker(id="alice", hook=hook)
bob = sy.VirtualWorker(id="bob", hook=hook)
james = sy.VirtualWorker(id="james", hook=hook)

In [0]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]])
target = torch.tensor([[0],[0],[1],[1.]])

# A Toy Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x
model = Net()

In [0]:
# We encode everything
data = data.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)
target = target.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)
model = model.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)

In [132]:
print(data)

(Wrapper)>AutogradTensor>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:28790545884 -> bob:4820121118]
	-> [PointerTensor | me:33515571378 -> alice:83089053615]
	*crypto provider: james*


In [133]:
opt = optim.SGD(params=model.parameters(),lr=0.1).fix_precision()

for iter in range(20):
    # 1) erase previous gradients (if they exist)
    opt.zero_grad()
    # 2) make a prediction
    pred = model(data)
    # 3) calculate how much we missed
    loss = ((pred - target)**2).sum()
    # 4) figure out which weights caused us to miss
    loss.backward()
    # 5) change those weights
    opt.step()
    # 6) print our progress
    print(loss.get().float_precision())

tensor(1.1860)
tensor(1.0080)
tensor(1.0010)
tensor(0.9990)
tensor(0.9980)
tensor(0.9960)
tensor(0.9960)
tensor(0.9950)
tensor(0.9940)
tensor(0.9920)
tensor(0.9890)
tensor(0.9870)
tensor(0.9840)
tensor(0.9800)
tensor(0.9750)
tensor(0.9720)
tensor(0.9660)
tensor(0.9580)
tensor(0.9510)
tensor(0.9420)


In [0]:
%reset -f